In [ ]:
##############################Imports##############################
%matplotlib inline
from __future__ import print_function

import tensorflow as tf # version 2.7.0
import tensorflow_pro$bability as tfp # version 0.15.0
import numpy as np
import random as rd

import matplotlib.pyplot as plt
from matplotlib import cm

from mpl_toolkits import mplot3d

import timeit
import time
from scipy import sparse
import scipy.stats as stats
from scipy.stats import norm as nm
import scipy.interpolate as spi
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp
from scipy.linalg import norm
from scipy.linalg import solve
from IPython.display import display
import sympy; sympy.init_printing()
def display_matrix(m):
    display(sympy.Matrix(m))


# Content
There are 4 main parts in our final project.

* Introduction

* Discretizing the Black Scholes Equation
  * Grid
  * Boundary Conditions

* European Option - Numerical Methods for Solving the BS BVP
  * Backward Euler Method
  * Successive Over-Relaxation Method
  * Crank-Nicholson Method
  * Monte Carlo Method
  * Cross-Comparison of Methods
* American Option - Numerical Methods for Solving the BS BVP
  * Assumptions and Boundary Conditions
  * SOR-Crank-Nicholson
  * Longstaff-Shortz (Monte Carlo method)
  * Comparision of Methods


## True Black Scholes Solution

In [ ]:
# ---------------------------- True BS Solution ----------------------------

def BlkSos(r, S, K, T, sigma, type = "Call"):
    """Calculate True BlkSos solution
    input: 
    r = risk_free_rate_of_return
    S = stock price
    K = strike price
    T = T expiry
    sigma = implied_volatility
    type = "Call" or "Put"
    
    
    output: price
    """
    d1 = (np.log(S/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if type == "Call":
        Price = S * nm.cdf(d1, 0, 1) - K * np.exp(-r * T) * nm.cdf(d2, 0, 1)

    elif type == "Put":
        Price = K * np.exp(-r * T) * nm.cdf(-d2, 0, 1) - S * nm.cdf(-d1, 0, 1)

    return Price

## Parameters Setup

In [ ]:
# ---------------------------- Parameter Setup ----------------------------
r = 0.06         # risk_free_rate_of_return
sigma = 0.3      # implied_volatility
S_0 = 5          # initial stock price
X_0 = np.log(50)
K = 5            # strike price
T_end = 1        # T expir

# Space/Time steps

N_space = 500 
N_time = 500 

S_max = 3*float(K)                
S_min = float(K)/3

x_max = np.log(S_max)  
x_min = np.log(S_min) 

# Space/Time Discretization

x, dx = np.linspace(x_min, x_max, N_space, retstep=True)   
T, dt = np.linspace(0, T_end, N_time, retstep=True)  

# Call Payoff
payoff = np.maximum(np.exp(x)-K,0)    

# Grid N_space and N_time
V_GS = np.zeros((N_space,N_time)) 
V_SOR = np.zeros((N_space,N_time)) 

# Boundary
rhs = np.zeros(N_space-2)          

# Initial Value and Boundary Conditions
V_GS[:,-1] = payoff                  
V_GS[-1,:] = np.exp(x_max) - K * np.exp(-r* T )
V_GS[0,:] = 0    

# Initial Value and Boundary Conditions
V_SOR[:,-1] = payoff                  
V_SOR[-1,:] = np.exp(x_max) - K * np.exp(-r* T )
V_SOR[0,:] = 0  

# define the variables which apppear in the tri-diagonal matrix D and construct matrix D

a = ( (dt/2) * ( (r-0.5*(sigma**2))/dx - (sigma**2)/(dx**2) ) )
b = ( 1 + dt * ( (sigma**2)/(dx**2) + r ) )
c = (-(dt/2) * ( (r-0.5*(sigma**2))/dx + (sigma**2)/(dx**2) ) )

D = sparse.diags([a, b, c], [-1, 0, 1], shape=(N_space-2, N_space-2)).tocsc()

# In order to calculate the convergence
order_C = lambda delta_x, error, order: np.exp(np.log(error) - order * np.log(delta_x))

## Backward Euler Method

In [ ]:
# ---------------------------- Numerical Method Setup ----------------------------
# Backward
for i in range(N_time-2, -1, -1):
    rhs[0] = a * V_GS[0, i]
    rhs[-1] = c * V_GS[-1, i]
    V_GS[1:-1, i] = spsolve(D, (V_GS[1:-1,i+1] - rhs))

In [ ]:
# ---------------------------- Problem Solution and Plots ----------------------------
is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

S = np.exp(x)
plt.figure(figsize=(8,8))
plt.ion()
plt.plot(S, payoff, color='blue',label="Payoff")
plt.xlabel("Stock Prices")
plt.ylabel("Option Prices")
plt.legend(loc='best')
plt.title("BS Price of Backward")
for i in range(10):
    
    plt.plot(S, V_GS[:,int((9-i)*N_time/10)], color='r',label="BS curve of Backward")
#     plt.pause(1)
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())
    plt.draw()
    time.sleep(0.5)
# plt.show()
plt.clf()

fig = plt.figure(figsize=(17,8))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2, projection='3d')

ax1.plot(S, payoff, color='b',label="Payoff")
ax1.plot(S, V_GS[:,-1], color='r',label="BS Curve of Backward")

ax1.set_xlabel("Stock Prices")
ax1.set_ylabel("Option Prices")
ax1.legend(loc='best')
ax1.set_title("BS Price of Backward at t=0")

X, Y = np.meshgrid(T[::-1], S)
ax2.plot_surface(Y, X, V_GS, cmap=cm.ocean)
ax2.set_title("BS Price of Backward 3D Model")
ax2.set_xlabel("Stock Prices")
ax2.set_ylabel("T")
ax2.set_zlabel("Option Prices")
ax2.view_init(30, -135) 
plt.show()

## Successive Over-Relaxation Method

In [ ]:
# ---------------------------- Numerical Method (SOR) Setup ----------------------------
# SOR

def SOR(a, b, c, d, w=1, eps=1e-10, itera = 100):
    
    """SOR
    
    input:
        a:       Black Scholes Coresponding Parameters
        b:       Black Scholes Coresponding Parameters
        c:       Black Scholes Coresponding Parameters
        d:       the array we need to put in and calculate
        omega:   SOR parameter
        epsilon:     tolerance
        itera:   iteration
        
    output:
        x:       array calculated by SOR algorithm
        
    """
    

    N = len(d)

    # Initial Guess
    x_old = np.ones(N)  

    # New Solution             
    x_new = np.ones(N)               
    
    for k in range(itera):

        for i in range(N):
            
            if i == 0 :
                sor = c * x_new[1]
            elif (i == N-1):
                sor = a * x_new[N-2]
            else:
                sor = a * x_new[i-1] + c * x_new[i+1]

            x_new[i] = (1 - w) * x_new[i] + (w / b) * (d[i] - sor)  

        # if norm(x_new - x_old) < eps:

    return x_new

w = 2.0 / (1.0 + np.sin(np.pi * dx))

for i in range(N_time-2,-1,-1):
    if i % 100 == 0:
        print("Have already done {} %".format(round((1-i/N_time)*100)))  
    rhs[0] = a * V_SOR[0,i]
    rhs[-1] = c * V_SOR[-1,i]
    
    # European
    V_SOR[1:-1,i] =  SOR(a, b, c, (V_SOR[1:-1,i+1] - rhs), w=w, eps=1e-10, itera=300 )

    # # American
    # V_SOR[1:-1,i] = np.maximum( SOR( a,b,c, (V_SOR[1:-1,i+1] - rhs), w=w, eps=1e-10, N_max=300 ), rhs[1:-1]) 
    

In [ ]:
# ---------------------------- Problem Solution and Plots ----------------------------
is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

S = np.exp(x)
plt.figure(figsize=(8,8))
plt.ion()
plt.plot(S, payoff, color='blue',label="Payoff")
plt.xlabel("Stock Prices")
plt.ylabel("Option Prices")
plt.legend(loc='best')
plt.title("BS Price")
for i in range(10):

    plt.plot(S, V_SOR[:,int((9-i)*N_time/10)], color='red',label="BS curve")
#     plt.pause(1)
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())
    plt.draw()
    time.sleep(0.5)

plt.clf()

fig = plt.figure(figsize=(17,8))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2, projection='3d')

ax1.plot(S, payoff, color='b',label="Payoff")
ax1.plot(S, V_SOR[:,-1], color='r',label="BS Curve")
ax1.set_xlabel("Stock Prices")
ax1.set_ylabel("Option Prices")
ax1.legend(loc='best')
ax1.set_title("BS Price at t=0")

X, Y = np.meshgrid(T[::-1], S)
ax2.plot_surface(Y, X, V_SOR, cmap=cm.ocean)
ax2.set_title("BS Price 3D Model")
ax2.set_xlabel("Stock Prices")
ax2.set_ylabel("T")
ax2.set_zlabel("Option Prices")
ax2.view_init(30, -135) 
plt.show()

## Crank-Nicholson Method

In [ ]:
def LHS_matrix(M, alpha, beta, gamma):
    """generate and return the LHS coefficient matrix A.
    
    Arguments:
        M:       total number of spatials grids
        alpha:   array of coefficients on lower diagnoal
        beta:    array of coefficients on diagnoal
        gamma:   array of coefficients on upper diagnoal
    
    Returns:
        A:       LHS coefficient matrix
    """
    # diagonal
    d = np.diag(1+beta)
    # upper diagonal
    ud = np.diag(gamma[:-1], 1)
    # lower diagonal
    ld = np.diag(alpha[1:], -1)
    
    A = d + ud +ld
    return A

In [ ]:
def RHS(C, alpha, beta, gamma, S_max, K):
    """generate and return the RHS vector b.
    
    Arguments:
        C:       array of the price of call option at previous time step
        alpha:   array of coefficients on lower diagnoal
        beta:    array of coefficients on diagnoal
        gamma:   array of coefficients on upper diagnoal
        S_max:   upper bound of stock price
        E:       exercise price
    
    Returns:
        b:       RHS vector
    """
    # diagonal of A_star
    d = np.diag(1-beta)
    # upper diagonal of A_star
    ud = np.diag(-gamma[:-1], 1)
    # lower diagonal of A_star
    ld = np.diag(-alpha[1:], -1)
    
    A_star = d + ud + ld
    b = np.dot(A_star,C[1:-1])
    # add BC for the right bound (the last element)
    b[-1] += -2*gamma[-1] * (S_max-K) 
    
    return b

In [ ]:
def CrankNicolson(C, A, N, alpha, beta, gamma, S_max, K):
    """using Crank-Nicolson scheme to solve the Black-Scholes equation for the call option price.
    
    Arguments:
        C:       array of the price of call option
        A:       LHS coefficient matrix
        N:       total number of time steps       
        alpha:   array of coefficients on lower diagnoal
        beta:    array of coefficients on diagnoal
        gamma:   array of coefficients on upper diagnoal
        S_max:   upper bound of stock price
        E:       exercise price
    
    Returns:
        C:       array of the price of call option
    """
    for t in range(N):
        b = RHS(C, alpha, beta, gamma, S_max, K)
        # use numpy.linalg.solve
        C[1:-1] = solve(A,b)
    return C

In [ ]:
def CN_iterations(N, M, T, r, sigma, K):
    """
      T:                 expiry time
      r =                no-risk interest rate
      sigma =            volatility of underlying asset
      E                  exercise price (sometimes known as K)
             upper bound of price of the stock (4*E)
      N                  number of time steps 
      M                  number of space grids
    """

    dt = T/N       # time step
    s = np.linspace(S_min, S_max, M+1)   # spatial grid (stock's price)

    # initial condition & boundary condition
    C = s - K
    C = np.clip(C, 0, S_max-K)


    # calculating the coefficient arrays
    index = np.arange(1,M)

    alpha = dt/4 * (r*index - sigma**2*index**2)
    beta = dt/2 * (r + sigma**2*index**2)
    gamma = -dt/4 * (r*index + sigma**2*index**2)

    A = LHS_matrix(M, alpha, beta, gamma)
    C_imp = CrankNicolson(C, A, N, alpha, beta, gamma, S_max, K)
    
    return s, C_imp

In [ ]:
s,C_imp = CN_iterations(N=500, M=500, T = 0, r=r, sigma=sigma, K=K)
plt.figure(figsize=(8,8))
plt.plot(s,C_imp,color='#cd3333', ls='--', lw=3, label='Crank-Nicolson')
plt.plot(s, BlkSos(r, s, K , 0, sigma), label='True Black-Scholes')
plt.xlabel("Stock Price")
plt.ylabel("Option price")
plt.legend()

## Monte Carlo Method

In [ ]:
#Formula Variables / Parameters chosen
stock_price = S_0
strike_price = K
time_to_expire = T_end
implied_volitility = sigma
risk_free_rate_of_return = r
seed = int(150*rd.random()) #randomly selected

#Path Simulator Graph - setup code

def black_scholes_formula(stockPrice, strike, time_to_expr, imp_vol, risk_free, timedelta, stdnorm_random_variates):
    return stockPrice * tf.math.cumprod(tf.exp((risk_free - 0.5 * imp_vol ** 2) * timedelta + imp_vol * tf.sqrt(timedelta) * stdnorm_random_variates), axis = 1)

def make_path_simulator(stockPrice, strike, time_to_expr, imp_vol, risk_free, seed, n_sims, obs):
    #Create the time variables
    if seed != 0:
        np.random.seed(seed)
    stdnorm_random_variates = np.random.randn(n_sims, obs)
    timedelta = time_to_expr / stdnorm_random_variates.shape[1]
    return black_scholes_formula(stockPrice, strike, 
                                 time_to_expr, imp_vol, risk_free, timedelta, stdnorm_random_variates)

In [ ]:
#Path Simulator Graph - graphing code
path_simulator = tf.function(make_path_simulator)
paths = path_simulator(stock_price,strike_price,
                       time_to_expire,implied_volitility,risk_free_rate_of_return, seed, 10, 400)
plt.figure(figsize=(8,5))
plt.plot(np.transpose(paths))
plt.title('Simulated Path')
plt.ylabel('Price')
plt.xlabel('TimeStep')
plt.show()

In [ ]:
def create_montecarlo_tf_pricer(stockPrice, strike, time_to_expr, imp_vol, risk_free, seed, n_sims, obs):
    if seed != 0:
        np.random.seed(seed)
    
    stdnorm_random_variates = np.random.randn(n_sims, 1)
    time_del = time_to_expr / stdnorm_random_variates.shape[1]
    
    S_T = black_scholes_formula(stockPrice, strike, time_to_expr, imp_vol, risk_free, time_del, stdnorm_random_variates)
    payout = tf.maximum(S_T[:,-1] - strike, 0)
    npv = tf.exp(-risk_free*time_to_expr) * tf.reduce_mean(payout)
    target_calc = npv
    return target_calc

In [ ]:
sims = 1000
observations = 10

montecarlo_tf_pricer = tf.function(create_montecarlo_tf_pricer)

In [ ]:
s_2=s.tolist()
prices=[]
for s_i in s_2:
    mon_ter=montecarlo_tf_pricer(s_i,strike_price,time_to_expire,implied_volitility,risk_free_rate_of_return, seed, sims, 2000).numpy()
    prices.append(mon_ter)

## Cross-Comparison of Methods

### Comparision of 4 Different Methods when t = 0

In [ ]:
# ---------------------------- Comparison of 4 Different Methods when t = 0 ----------------------------

true_price = []

# Calculating the true values and compare
for i in range(N_space):
    true_price.append(BlkSos(r, np.exp(x[i]), K, 0, sigma, type = "Call"))

plt.figure(figsize=(8,8))

plt.plot(S, true_price, 'r', label="BS True Solution", markersize = 0.5) 
plt.plot(S, V_SOR[:,-1], 'y', label="BS Curve of SOR", markersize = 0.5)  
plt.plot(s,C_imp,color='#cd3333', ls='--', lw=3, label='Crank-Nicolson')
plt.plot(s,prices,color='orange', marker='+', lw=3, label='Monte Carlo')  
plt.plot(S, V_GS[:,-1], 'b',label="BS Curve of Backward", markersize = 0.5)


plt.title("Comparision of 4 Different Methods when t = 0")
plt.ylabel("Option Prices")
plt.xlabel("Stock Prices")

plt.legend(loc='best')
plt.show()


### Convergence Plot of SOR (wrt Iteration)

In [ ]:
# ---------------------------- Convergence Plot of SOR (wrt iteration) ----------------------------

# number of iteration
iteration = [10, 300, 500, 1000]

plt.figure(figsize=(8,8))

true_price = np.zeros(len(iteration))

# Setup an enpty list in order to contain the difference between true value and the calculation result
diff_set = []
for tt, iter in enumerate(iteration):

    # Grid N_space and N_time
    V_SOR = np.zeros((N_space,N_time)) 

    # Initial Value and Boundary Conditions of SOR
    V_SOR[:,-1] = payoff                  
    V_SOR[-1,:] = np.exp(x_max) - K * np.exp(-r* T[::-1] )
    V_SOR[0,:] = 0  
    
    for i in range(N_time-2,-1,-1):
        if i % 100 == 0:
            print("Number of Iteration: {}. Processed {} %".format(iter, round((1-i/N_time)*100)), end='')  
    
        rhs[0] = a * V_SOR[0,i]
        rhs[-1] = c * V_SOR[-1,i]
        V_SOR[1:-1,i] =  SOR(a, b, c, (V_SOR[1:-1,i+1] - rhs), w=w, eps=1e-10, itera=iter )
    
    V_SOR = np.fliplr(V_SOR)
    
    # Set the difference between true solution and numerical solution equals to 0
    
    diff = 0
    for k in range(N_time):
        
        blksos = BlkSos(r, np.exp(x), K, T[k], sigma, type = "Call")
        
        # Since some of the true value will be 1e-315, then the difference between true and calculated values will be super lagre
        # We then set anything smaller than 1e-10 as 0.
        blksos[blksos < 1e-10] = 0.

        diff += norm(blksos - V_SOR[:,k])

    diff_set.append(diff)

plt.title("Convergence of SOR wrt number of iteration")
plt.loglog(iteration, np.array(diff_set)/(N_time*N_space) , markersize = 1, label = "Convergence Curve")
plt.ylabel("$||True - SOR||$")
plt.xlabel("Number of Iteration")
plt.legend(loc = 'best')
plt.show()

### Convergence Plot of SOR (wrt delta_t, 3 schemes)

In [ ]:
# ---------------------------- Convergence Plot of SOR (wrt delta t and three schemes) ----------------------------

# Grid number of time
SOR_time=[200, 500, 900, 1500]
delta_t = np.array([1.0 / t for t in SOR_time])

# Three Schemes
r_set = [0.06, 0.06, 0.6 ]
sigma_set = [0.3, 0.03, 0.3 ]

# Create an array to store the difference
diff_set = np.zeros((len(SOR_time), len(r_set)))

plt.figure(figsize=(28,8))

for i_sc in range(3):
    for idx, sor_time in enumerate(SOR_time):
        
        
        # Delta_t Calculation
        T, dt = np.linspace(0, T_end, sor_time, retstep=True)  

        # Define the Variables which Apppear in the Tri-diagonal Matrix D and Construct Matrix D

        a = ( (dt/2) * ( (r_set[i_sc]-0.5*(sigma_set[i_sc]**2))/dx - (sigma_set[i_sc]**2)/(dx**2) ) )
        b = ( 1 + dt * ( (sigma_set[i_sc]**2)/(dx**2) + r_set[i_sc] ) )
        c = (-(dt/2) * ( (r_set[i_sc]-0.5*(sigma_set[i_sc]**2))/dx + (sigma_set[i_sc]**2)/(dx**2) ) )

        D = sparse.diags([a, b, c], [-1, 0, 1], shape=(N_space-2, N_space-2)).tocsc()

        # Set the difference between true solution and numerical solution equals to 0
        diff = 0

        # Grid N_space and sor_time
        V_SOR = np.zeros((N_space, sor_time)) 

        # Initial Value and Boundary Conditions of SOR
        V_SOR[:,-1] = payoff                  
        V_SOR[-1,:] = np.exp(x_max) - K * np.exp(-r_set[i_sc] * T[::-1] )
        V_SOR[0,:] = 0  
        
        for i in range(sor_time-2,-1,-1):

            # Monitor the process
            if i % 100 == 0:
                print("\r Scheme: {}, Delta t = {}, Processed {} %".format(i_sc+1, round(dt, 5), round((1-i/sor_time)*100)),end = '')  
        
            rhs[0] = a * V_SOR[0,i]
            rhs[-1] = c * V_SOR[-1,i]
            V_SOR[1:-1,i] =  SOR(a, b, c, (V_SOR[1:-1,i+1] - rhs), w=1.5, eps=1e-10, itera = 100 )
        
        V_SOR = np.fliplr(V_SOR)
        
    
        # Choose a fixed time and calculate the true value
        blksos = BlkSos(r, np.exp(x), K, T[int(0.5*sor_time)], sigma, type = "Call")


        diff += norm(blksos - V_SOR[:,int(0.5*sor_time)])/(norm(blksos)*N_space)

        diff_set[idx, i_sc] = diff

    plt.subplot(1, 3, i_sc+1)
    plt.loglog(delta_t, order_C(delta_t[0], diff_set[0,i], 1.0) * delta_t**1.0, '--', label="1st Order")
    plt.loglog(delta_t, diff_set[:,i_sc],'ro-', label="Scheme: {}, r = {}, sigma = {}".format(i_sc+1, r_set[i_sc], sigma_set[i_sc]))
    plt.title("Convergence of SOR wrt Delta t ")
    plt.ylabel("$||True - CN||$")
    plt.xlabel("Grid Number of Time")
    plt.legend(loc = 'best')
        

plt.show()

### Convergence Plot of C-N (wrt delta_t, 3 schemes)

In [ ]:
# ---------------------------- Convergence Plot of C-N (wrt delta t and three schemes) ----------------------------

# Grid number of time
CN_time=[100, 200, 400, 800]
delta_t = np.array([1.0 / t for t in CN_time])

# Three Schemes
r_set = [0.06, 0.06, 0.6 ]
sigma_set = [0.3, 0.03, 0.3 ]

# Create an array to store the difference
diff_set = np.zeros((len(CN_time), len(r_set)))

plt.figure(figsize=(28,8))

for i in range(3):
    for idx, cn_time in enumerate(CN_time):

        diff = 0.

        # Monitor the process

        print("\r Scheme: {}. Delta t = {} ".format(i+1, round(delta_t[idx],5)),end = '')  

        # Time Discritization for CN Convergence
        T_cn = np.linspace(0,1,cn_time)

        # Result of CN_iteration
        s, C_imp = CN_iterations(cn_time, N_space, T_cn[int(0.5*cn_time)], r_set[i], sigma_set[i], K)

        # True Value
        true = BlkSos(r_set[i],s,K,T_cn[int(0.5*cn_time)],sigma_set[i],type="Call")

        # Difference between CN_iteration and True Value
        diff = norm(C_imp-true, ord = 1)/norm(true)
        diff_set[idx,i] = diff/N_space

    plt.subplot(1, 3, i+1)
    plt.loglog(CN_time, diff_set[:,i],"ro-", label="Scheme: {}, r = {}, sigma = {}".format(i+1, r_set[i], sigma_set[i]))
    plt.title("Convergence of Crank-Nicholson wrt Delta t ")
    plt.ylabel("$log(||True - CN||)$")
    plt.xlabel("log(N)")
    plt.legend(loc = 'best')

plt.show()
        

###  Convergence Plot of all 4 methods (wrt delta t)

In [ ]:
# ---------------------------- Convergence Plot of all 4 methods (wrt delta t) ----------------------------

# Grid number of time
iter_time=[200, 500, 900, 1500]
delta_t = np.array([1.0 / t for t in iter_time])

# Create an array to store the difference
diff_set = np.zeros((len(iter_time), 4))


for idx, iter_time in enumerate(iter_time):
    
    
    # Delta_t Calculation
    T, dt = np.linspace(0, T_end, iter_time, retstep=True)  

    # Define the Variables which Apppear in the Tri-diagonal Matrix D and Construct Matrix D

    a = ( (dt/2) * ( (r-0.5*(sigma**2))/dx - (sigma**2)/(dx**2) ) )
    b = ( 1 + dt * ( (sigma**2)/(dx**2) + r ) )
    c = (-(dt/2) * ( (r-0.5*(sigma**2))/dx + (sigma**2)/(dx**2) ) )

    D = sparse.diags([a, b, c], [-1, 0, 1], shape=(N_space-2, N_space-2)).tocsc()

    # Set the difference between true solution and numerical solution equals to 0
    diff = 0

    # Grid N_space and iter_time
    V_SOR = np.zeros((N_space, iter_time)) 

    # Initial Value and Boundary Conditions of SOR
    V_SOR[:,-1] = payoff                  
    V_SOR[-1,:] = np.exp(x_max) - K * np.exp(-r * T[::-1] )
    V_SOR[0,:] = 0  

    
    for i in range(iter_time-2,-1,-1):

        # Monitor the process
        if i % 100 == 0:
            print("\r SOR: Delta t = {}, Processed {} %".format(round(dt, 5), round((1-i/iter_time)*100)),end = '')  
    
        rhs[0] = a * V_SOR[0,i]
        rhs[-1] = c * V_SOR[-1,i]
        V_SOR[1:-1,i] =  SOR(a, b, c, (V_SOR[1:-1,i+1] - rhs), w=1.5, eps=1e-10, itera = 100 )
    
    V_SOR = np.fliplr(V_SOR)
    
    print('\r Start processing GS')
    # Grid N_space and N_time
    V_GS = np.zeros((N_space,iter_time)) 


    # Boundary
    rhs = np.zeros(N_space-2)          

    # Initial Value and Boundary Conditions
    V_GS[:,-1] = payoff                  
    V_GS[-1,:] = np.exp(x_max) - K * np.exp(-r* T )
    V_GS[0,:] = 0  

    # Backward
    for i in range(iter_time-2, -1, -1):
        rhs[0] = a * V_GS[0, i]
        rhs[-1] = c * V_GS[-1, i]
        V_GS[1:-1, i] = spsolve(D, (V_GS[1:-1,i+1] - rhs))

    V_GS = np.fliplr(V_GS)
    
    print('\r Start processing C-N')
    # C-N
    s,C_imp = CN_iterations(N=iter_time, M=N_space-1, T = int(0.5*iter_time), r=r, sigma=sigma, K=K)

    print('\r Start processing M-C')
    # Monte Carlo
    s_2=s.tolist()
    prices=[]
    for s_i in s_2:
        mon_ter=montecarlo_tf_pricer(s_i,strike_price,time_to_expire,implied_volitility,risk_free_rate_of_return, seed, sims, 2000).numpy()
        prices.append(mon_ter)
    
    # Choose a fixed time and calculate the true value
    blksos = BlkSos(r, np.exp(x), K, T[int(0.5*iter_time)], sigma, type = "Call")
  

    diff_set[idx, 0] = norm(blksos - V_SOR[:,int(0.5*iter_time)])/(norm(blksos)*N_space)
    diff_set[idx, 1] = norm(blksos - V_GS[:,int(0.5*iter_time)])/(norm(blksos)*N_space)
    diff_set[idx, 2] = norm(blksos - C_imp)/(norm(blksos)*N_space)
    diff_set[idx, 3] = norm(blksos - prices)/(norm(blksos)*N_space)

plt.figure(figsize=(18,18))

plt.subplot(2, 2, 1)
plt.loglog(delta_t, order_C(delta_t[0], diff_set[0,0], 1.0) * delta_t**1.0, '--', label="1st Order")
plt.loglog(delta_t, diff_set[:,0],'ro-', label="GS")
plt.title("Convergence of GS wrt Delta t ")
plt.ylabel("$||True - CN||$")
plt.xlabel("Delta t")
plt.legend(loc = 'best')
        
plt.subplot(2, 2, 2)
plt.loglog(delta_t, order_C(delta_t[0], diff_set[0,1], 1.0) * delta_t**1.0, '--', label="1st Order")
plt.loglog(delta_t, diff_set[:,1],'ro-', label="SOR")
plt.title("Convergence of SOR wrt Delta t ")
plt.ylabel("$||True - CN||$")
plt.xlabel("Delta t")
plt.legend(loc = 'best')

plt.subplot(2, 2, 3)
plt.loglog(delta_t, order_C(delta_t[0], diff_set[0,2], 1.0) * delta_t**1.0, '--', label="1st Order")
plt.loglog(delta_t, diff_set[:,2],'ro-', label="C-N")
plt.title("Convergence of C-N wrt Delta t ")
plt.ylabel("$||True - CN||$")
plt.xlabel("Delta t")
plt.legend(loc = 'best')

plt.subplot(2, 2, 4)
plt.loglog(delta_t, order_C(delta_t[0], diff_set[0,3], 1.0) * delta_t**1.0, '--', label="1st Order")
plt.loglog(delta_t, diff_set[:,3],'ro-', label="M-C")
plt.title("Convergence of M-C wrt Delta t ")
plt.ylabel("$||True - CN||$")
plt.xlabel("Delta t")
plt.legend(loc = 'best')

plt.show()

### comparision of running time

In [ ]:
# Calculate the backward run time

%%time 
# ---------------------------- Numerical Method Setup ----------------------------
# Backward
for i in range(N_time-2, -1, -1):
    rhs[0] = a * V_GS[0, i]
    rhs[-1] = c * V_GS[-1, i]
    V_GS[1:-1, i] = spsolve(D, (V_GS[1:-1,i+1] - rhs))

In [ ]:
# Calculate the SOR run time

%%time 
# Grid N_space and N_time
V_SOR = np.zeros((N_space,N_time)) 

# Initial Value and Boundary Conditions of SOR
V_SOR[:,-1] = payoff                  
V_SOR[-1,:] = np.exp(x_max) - K * np.exp(-r* T[::-1] )
V_SOR[0,:] = 0  

for i in range(N_time-2,-1,-1):
    if i % 100 == 0:
        print("Number of Iteration: {}. Processed {} %".format(iter, round((1-i/N_time)*100)), end='')  

    rhs[0] = a * V_SOR[0,i]
    rhs[-1] = c * V_SOR[-1,i]
    V_SOR[1:-1,i] =  SOR(a, b, c, (V_SOR[1:-1,i+1] - rhs), w=w, eps=1e-10, itera=300)

In [ ]:
# Calculate the C-N run time

%%time

for i in range(500):
    s,C_imp = CN_iterations(N=500, M=500, T = T[i], r=r, sigma=sigma, K=K)

In [ ]:
# Calculate the Monte run time

%%time
s_2=s.tolist()
prices=[]
for s_i in s_2:
    mon_ter=montecarlo_tf_pricer(s_i,strike_price,time_to_expire,implied_volitility,risk_free_rate_of_return, seed, sims, 2000).numpy()
    prices.append(mon_ter)

# American Option

# SOR 

In [ ]:
# ---------------------------- Parameter Setup ----------------------------
#Blk-Sos Parameters
r = 0.06
sigma = 0.3
S_0 = 5
X_0 = np.log(50)
K = 5
T_end = 1

# Space/Time steps

N_space = 500 
N_time = 500 

S_max = 3*float(K)                
S_min = float(K)/3

x_max = np.log(S_max)  
x_min = np.log(S_min) 

# Space/Time Discretization
x, dx = np.linspace(x_min, x_max, N_space, retstep=True)   
T, dt = np.linspace(0, T_end, N_time, retstep=True)  

# Call Payoff
payoff = np.maximum(np.exp(x)-K,0)    

# Boundary
rhs = np.zeros(N_space-2)      

In [ ]:
def American_SOR(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, alpha, epsilon):
    """SOR
    
    :input:
        S:       asset price
        K:       strike price
        T:       time of expiry
        r:       risk free interest rate
        q: 
        sigma:   volatility
        
        option_type: 'call' or 'put'
        Smin:        min S for grid
        Smax:        max S for grid
        Ns:          Number of space points
        Nt:          Number of time points
        theta:       for convergence
        alpha:       constant coefficient 
        epsilon:     tolerance
        
    :output:
        x:       s-grid
        t:       t-grid
        V_SOR:   option prices at every point in the (x,t) grid
        
    
    """
    
    #setting up some coefficients
    if option_type=='call':
        omega = 1 
    else:
        omega =-1
    
    Ns = int(Ns)
    max_iter = 10*Nt
    Nt = int(Nt)
    dS = (Smax-Smin)/Ns * 1.0
    dt = T/Nt*1.0
    S_space = np.linspace(Smin, Smax, Ns+1)
    T_space = np.linspace(0, T, Nt+1)
    grid = np.zeros(shape=(Ns+1, Nt+1))
    
    # boundary_conditions:
    tau = T_space[-1] - T_space;     
    
    # american option boundary condition (from new PDE as described)
    grid[0,  :] = np.maximum(omega*(S_space[0]  - K), 0)
    grid[-1, :] = np.maximum(omega*(S_space[-1] - K), 0)     
    grid[:, -1] = np.maximum(omega*(S_space - K), 0)
    
    #setting up more coefficients
    drift = (r-q)*S_space[1:-1]/dS
    diffusion_square = (sigma*S_space[1:-1]/dS)**2

    l = 0.5*(diffusion_square - drift)
    c = -diffusion_square - r
    u = 0.5*(diffusion_square + drift)
    
    
    #setting up matrix
    A = sp.diags([l[1:], c, u[:-1]], [-1, 0, 1],  format='csc')
    I = sp.eye(Ns-1)
    M1 = I + (1-theta)*dt*A
                  
    w = alpha
    thedt = theta * dt
    payoff = grid[1:-1, -1]
    m = len(payoff)
    previous_step = payoff.copy()
    
    def trigger( oldval, new_step, tol, counter, max_iteration ):
        no_break = 1
        if np.max( np.abs(new_step-oldval)/np.maximum(1,np.abs(new_step)) ) <= tol:
            no_break = 0
        elif counter > max_iteration:
            print('The result may not converge.')
            no_break = 0
        return no_break
        
    for j in reversed(np.arange(Nt)):
        counter = 0
        no_break = 1
        new_step = previous_step.copy()

        z = M1.dot(previous_step)

        z[0] += theta*l[0]*dt*grid[0, j] \
             + (1-theta)*l[0]*dt*grid[0, j+1] 
        z[-1] += theta*u[-1]*dt*grid[-1, j] \
              + (1-theta)*u[-1]*dt*grid[-1, j+1] 

        while no_break:
            counter += 1
            oldval = new_step.copy()
            new_step[0] = np.maximum( payoff[0], oldval[0] + w/(1-thedt*c[0]) \
                                   *( z[0] - (1-thedt*c[0])*oldval[0] \
                                     + thedt*u[0]*oldval[1]) )
            for k in np.arange(1,m-1):
                new_step[k] = np.maximum(payoff[k], oldval[k] + w/(1-thedt*c[k]) \
                                       *( z[k] + thedt*l[k]*new_step[k-1] \
                                         - (1-thedt*c[k])*oldval[k] \
                                         + thedt*u[k]*oldval[k+1]) )

            new_step[m-1] = np.maximum( payoff[m-1], oldval[m-1] + w/(1-thedt*c[m-1]) \
                                     *( z[m-1] + thedt*l[m-1]*new_step[m-2] \
                                       - (1-thedt*c[m-1])*oldval[m-1]) )

            no_break = trigger( oldval, new_step, epsilon, counter, max_iter )

        previous_step = new_step.copy()
        grid[1:-1, j] = previous_step

    return spi.splev( S, tck = spi.splrep( S_space, grid[:,0], k=3 ) )

In [ ]:
# -----------testing with specific initial conditions -------------
(S, K, r, q, T, sigma, option_type) = (12, 5, 0.1, 0.01, 1, 0.4, 'call')
(theta, alpha, epsilon) = (0.5, 1.5, 1e-6)

(Smin, Smax, Ns, Nt) = (0, 4*np.maximum(8,K), 400, 400)

amer_opt = American_SOR(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, alpha, epsilon)
print(amer_opt)

In [ ]:
#---------Function for Generating Comparison of SOR vs BS original Equation-------------
def SOR_prices(K,T,r):
    """ For a series of parameters schemes, compare the SOR- American Version
    price with the closed form European call price. Hold K,T,r constant
    :params:
        K: strike price
        T: end time
        r: risk free interest rate
        
    :returns:
        table_values: lists of tuples (S0, sigma, T, AmericanPutMC, true)    
    """
    table_values=[]
    AM_SOR_values=[]
    for S0 in (2., 5., 8.,):  # initial stock price values
        for vol in (0.2,0.4):  # sigma/volatility values
            for T in (1.0, 2.0):  # times-to-maturity
                
                (Smin, Smax, Ns, Nt) = (0, 4*np.maximum(8,K), 400, 400)
                AM_SOR= American_SOR(S0, K, r, q, T, vol, 'call', Smin, Smax, Ns, Nt, theta, alpha, epsilon)
                AM_SOR_values.append(AM_SOR)    
                
                true = BlkSos(r,S0, K, T, vol,)
                table_values.append((S0, vol, T, AM_SOR, true))
                print ("S0: %f, K:%f, T:%f, r:%f, sigma:%f--True:%f--AM_SOR:%f--Diff%f" % (S0, K, T, r, vol, true, AM_SOR, true-AM_SOR))
    return table_values, AM_SOR_values

In [ ]:
#----------Generating Comparison of American SOR vs European BS Original Equation-----------
K=5
T=1
r=0.1

t0 = time.time()
SOR_tables_vals = SOR_prices(K,T,r)
t1 = time()
d1 = t1 - t0
print ("Duration in Seconds %6.3f" % d1)

# Monte Carlo Least Squares

In [ ]:
#-----------------Lonstaff Schwartz Least Squares Monte Carlo Approach-------------
#Code Attribution: Jesus Perez Colino
#Accessed via https://github.com/jpcolino/IPython_notebooks/blob/master/Least%20Square%20Monte%20Carlo%20Implementation%20in%20a%20Python%20Class.ipynb


class AmericanOptionsLSMC(object):
    """ Class for American options pricing using Longstaff-Schwartz (2001):
    "Valuing American Options by Simulation: A Simple Least-Squares Approach."
    Review of Financial Studies, Vol. 14, 113-147.
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    M : int : grid or granularity for time (in number of total points)
    r : float : constant risk-free short rate
    div :    float : dividend yield
    sigma :  float : volatility factor in diffusion term 
    
    Unitest(doctest): 
    >>> AmericanPUT = AmericanOptionsLSMC('put', 36., 40., 1., 50, 0.06, 0.06, 0.2, 10000  )
    >>> AmericanPUT.price
    4.4731177017712209
    """

    def __init__(self, option_type, S0, strike, T, M, r, div, sigma, simulations):
        try:
            self.option_type = option_type
            assert isinstance(option_type, str)
            self.S0 = float(S0)
            self.strike = float(strike)
            assert T > 0
            self.T = float(T)
            assert M > 0
            self.M = int(M)
            assert r >= 0
            self.r = float(r)
            assert div >= 0
            self.div = float(div)
            assert sigma > 0
            self.sigma = float(sigma)
            assert simulations > 0
            self.simulations = int(simulations)
        except ValueError:
            print('Error passing Options parameters')


        if option_type != 'call' and option_type != 'put':
            raise ValueError("Error: option type not valid. Enter 'call' or 'put'")
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError('Error: Negative inputs not allowed')

        self.time_unit = self.T / float(self.M)
        self.discount = np.exp(-self.r * self.time_unit)

    @property
    def MCprice_matrix(self, seed = 123):
        """ Returns MC price matrix rows: time columns: price-path simulation """
        np.random.seed(seed)
        MCprice_matrix = np.zeros((self.M + 1, self.simulations), dtype=np.float64)
        MCprice_matrix[0,:] = self.S0
        for t in range(1, self.M + 1):
            brownian = np.random.standard_normal( int(self.simulations / 2))
            brownian = np.concatenate((brownian, -brownian))
            MCprice_matrix[t, :] = (MCprice_matrix[t - 1, :]
                                  * np.exp((self.r - self.sigma ** 2 / 2.) * self.time_unit
                                  + self.sigma * brownian * np.sqrt(self.time_unit)))
        return MCprice_matrix

    @property
    def MCpayoff(self):
        """Returns the inner-value of American Option"""
        if self.option_type == 'call':
            payoff = np.maximum(self.MCprice_matrix - self.strike,
                           np.zeros((self.M + 1, self.simulations),dtype=np.float64))
        else:
            payoff = np.maximum(self.strike - self.MCprice_matrix,
                            np.zeros((self.M + 1, self.simulations),
                            dtype=np.float64))
        return payoff

    @property
    def value_vector(self):
        value_matrix = np.zeros_like(self.MCpayoff)
        value_matrix[-1, :] = self.MCpayoff[-1, :]
        for t in range(self.M - 1, 0 , -1):
            regression = np.polyfit(self.MCprice_matrix[t, :], value_matrix[t + 1, :] * self.discount, 5)
            continuation_value = np.polyval(regression, self.MCprice_matrix[t, :])
            value_matrix[t, :] = np.where(self.MCpayoff[t, :] > continuation_value,
                                          self.MCpayoff[t, :],
                                          value_matrix[t + 1, :] * self.discount)

        return value_matrix[1,:] * self.discount


    @property
    def price(self): return np.sum(self.value_vector) / float(self.simulations)
    
    @property
    def delta(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        return (myCall_1.price - myCall_2.price) / float(2. * diff)
    
    @property
    def gamma(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        return (myCall_1.delta - myCall_2.delta) / float(2. * diff)
    
    @property
    def vega(self):
        diff = self.sigma * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma + diff, 
                                       self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0,
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma - diff, 
                                       self.simulations)
        return (myCall_1.price - myCall_2.price) / float(2. * diff)    
    
    @property
    def rho(self):        
        diff = self.r * 0.01
        if (self.r - diff) < 0:        
            myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r + diff, self.div, self.sigma, 
                                       self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
            return (myCall_1.price - myCall_2.price) / float(diff)
        else:
            myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r + diff, self.div, self.sigma, 
                                       self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r - diff, self.div, self.sigma, 
                                       self.simulations)
            return (myCall_1.price - myCall_2.price) / float(2. * diff)
    
    @property
    def theta(self): 
        diff = 1 / 252.
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T + diff, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T - diff, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
        return (myCall_2.price - myCall_1.price) / float(2. * diff)

In [ ]:
#---------Function for Generating Comparison of Monte Carlo vs BS original Equation---
def MC_prices(K,T,r):
    """ For a series of parameters schemes, compare the Monte Carlo Least Squares
    American call price with the closed form European call price
    :params:
        K: strike price
        T: end time
        r: risk free interest rate
        
    :returns:
        table_values: lists of tuples (S0, sigma, T, AmericanPutMC, true)
    
    """
    table_values = []
    for S0 in (2., 5., 8.,):  # initial stock price values
        for vol in (0.2, 0.4):  # volatility values
            for T in (1.0, 2.0):  # times-to-maturity
                AmericanCall = AmericanOptionsLSMC('call', S0, K, T, 50, r, 0, vol, 1500)
                true = BlkSos(r,S0, K, T, vol,)
                table_values.append((S0, vol, T, AmericanCall.price, true))
                
                print ("S0: %f, K:%f, T:%f, r:%f, sigma:%f--True:%f--LS MC:%f--Diff%f" % (S0, K, T, r, vol, true, float(AmericanCall.price), np.abs(true-AmericanCall.price)))
    return table_values                

In [ ]:
#----------Generating Comparison of Monte Carlo vs BS Original Equation-----------
K=5
T=1
r=0.1

t0 = time.time()
MC_tables_vals = MC_prices(K,T,r)
t1 = time.time()
d1 = t1 - t0
print ("Duration in Seconds %6.3f" % d1)

In [ ]:
#These are the values from the tables in the paper
#Re-running the above code should get the same values for SOR but not MC

SOR_vals = np.array([0.000003, 0.001196, 0.008405,  0.070752, 0.627691, 1.007536, 0.982618, 1.453893, 3.397119, 3.753795, 3.482719, 3.940458])
MC_vals = np.array([0.000000, 0.000336, 0.007982, 0.075290, 0.668399, 1.090520, 1.000716, 1.514198, 3.487210, 3.923219, 3.575221,  4.122688])

BS_vals = np.array([0.000003, 0.001454, 0.009010, 0.077005, 0.663484, 1.085968, 1.015923, 1.526494, 3.476583, 3.911142, 3.558124, 4.088889])

In [ ]:
relative_error = np.abs(np.array(SOR_vals)-np.array(MC_vals))/np.array(SOR_vals)
print(relative_error)